In [1]:
import pandas as pd
import sasoptpy as so
import os
import time

In [2]:
t0 = time.time()

In [3]:
df = pd.read_csv('../data/fpl_data.csv')
gk_data = df[df['position'] == 'Goalkeeper'].copy().reset_index()
gk_data.set_index('index', inplace = True)

In [4]:
def solve_goalkeeper_problem(gk_data, budget):
    model = so.Model(name='gk_model')
    players = gk_data.index.tolist()
    
    # Variables
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench',vartype=so.binary)
    
    # Objectives
    total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, 'value_season'] for p in players) + 0.1 * (so.expr_sum(bench[p] * gk_data.loc[p, 'value_season'] for p in players))
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')
    
    # Constraints
    model.add_constraints((lineup[p]+ bench[p] <= 1 for p in players), name='lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) ==1, name = 'single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) ==1, name = 'single_bench')
    model.add_constraint(so.expr_sum((lineup[p] + bench [p]) * gk_data.loc[p,'now_cost'] for p in players) <= budget, name='budget_con')
    
    # Solve Step
    model.export_mps(filename='gk.mps')
    command = 'cbc gk.mps solve solu solution.txt'
    # !{command} # jupyter notebook way of calling command to see log
    os.system(command) # using os way of calling command, to not display log
    
    with open('solution.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
        
        print("LINEUP")
        for p in players:
            if lineup[p].get_value() > 0.5:
                print(p,gk_data.loc[p])

        print("BENCH")
        for p in players:
            if bench[p].get_value() > 0.5:
                print(p,gk_data.loc[p])

In [8]:
solve_goalkeeper_problem(gk_data,88)

NOTE: Initialized model gk_model.
LINEUP
78 Unnamed: 0                     117
second_name            Raya Martin
team                     Brentford
element_type                     1
selected_by_percent           13.9
now_cost                        48
minutes                       2160
transfers_in               2007799
value_season                  22.7
total_points                   109
position                Goalkeeper
value                         22.7
Name: 78, dtype: object
BENCH
237 Unnamed: 0                    349
second_name                  Ward
team                    Leicester
element_type                    1
selected_by_percent          25.6
now_cost                       40
minutes                      2250
transfers_in              2312609
value_season                 22.2
total_points                   89
position               Goalkeeper
value                        22.2
Name: 237, dtype: object


In [6]:
t1 = time.time()
print(t1-t0, 'seconds')

0.47319531440734863 seconds
